In [2]:
import os
import re

label_dir = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\54_kule\detection_results_kamyon\kule_video_5\labels"

# 1) classes.txt'i atla, sadece data_<sayı>.txt dosyalarını işle
all_txts = [f for f in os.listdir(label_dir)
            if f.lower().endswith('.txt') and f.lower() != 'classes.txt']

# 2) data_<num>.txt formatını yakala, numaraya göre azalan sıraya diz
pat = re.compile(r'^data_(\d+)\.txt$', re.IGNORECASE)
files = []
for f in all_txts:
    m = pat.match(f)
    if m:
        files.append((int(m.group(1)), f))
    else:
        print(f"[UYARI] '{f}' ismi 'data_<num>.txt' formatına uymuyor, atlanıyor.")

files.sort(key=lambda x: x[0], reverse=True)

# 3) Çakışmayı önlemek için önce .tmp uzantısıyla taşı
for num, fname in files:
    old = os.path.join(label_dir, fname)
    new_num = num - 1
    if new_num < 1:
        print(f"[UYARI] '{fname}' için yeni numara {new_num} < 1, atlanıyor.")
        continue
    tmp_name = f"data_{new_num}.txt.tmp"
    os.rename(old, os.path.join(label_dir, tmp_name))

# 4) .tmp'leri gerçek isimlere çevir
for f in os.listdir(label_dir):
    if f.endswith('.tmp'):
        src = os.path.join(label_dir, f)
        dst = os.path.join(label_dir, f[:-4])  # .tmp kısmını parçalım
        os.rename(src, dst)

print("Label dosyaları başarıyla data_<i>.txt şeklinde yeniden numaralandırıldı.")


[UYARI] 'frame_000188.txt' ismi 'data_<num>.txt' formatına uymuyor, atlanıyor.
[UYARI] 'frame_000189.txt' ismi 'data_<num>.txt' formatına uymuyor, atlanıyor.
Label dosyaları başarıyla data_<i>.txt şeklinde yeniden numaralandırıldı.


In [3]:
import os
import re

label_dir = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\54_kule\detection_results_kamyon\kule_video_5\labels"  # gerekirse yolunuzu güncelleyin

# 1) classes.txt'i atla, sadece data_<sayı>.txt dosyalarını yakala
pat = re.compile(r'^data_(\d+)\.txt$', re.IGNORECASE)
to_fix = []
for fname in os.listdir(label_dir):
    m = pat.match(fname)
    if m and fname.lower() != 'classes.txt':
        num = int(m.group(1))
        if num >= 63:
            to_fix.append((num, fname))

# 2) Çakışmayı önlemek için en büyükten küçüğe sırala ve tmp ile taşı
to_fix.sort(key=lambda x: x[0], reverse=True)
for num, fname in to_fix:
    old_path = os.path.join(label_dir, fname)
    new_num = num - 1
    tmp_name = f"data_{new_num}.txt.tmp"
    os.rename(old_path, os.path.join(label_dir, tmp_name))

# 3) .tmp uzantılı dosyaları gerçek isimlere çevir
for f in os.listdir(label_dir):
    if f.endswith('.tmp'):
        src = os.path.join(label_dir, f)
        dst = os.path.join(label_dir, f[:-4])  # .tmp kısmını kaldır
        os.rename(src, dst)

print("data_63 ve sonrası için numaralar 1 azaltıldı.")


data_63 ve sonrası için numaralar 1 azaltıldı.


In [ ]:
#!/usr/bin/env python
"""
rename_yolo_pairs.py
Görüntü–etiket çiftlerini (YOLO) birlikte ve güvenli biçimde yeniden adlandırır.
"""

from pathlib import Path
import argparse
import shutil
import sys

VALID_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def collect_pairs(images_dir: Path, labels_dir: Path):
    """İki klasörü tarar, her görüntü-etiket çifti için (img_path, label_path) döndürür."""
    for img_path in sorted(p for p in images_dir.iterdir() if p.suffix.lower() in VALID_EXTS):
        label_path = labels_dir / (img_path.stem + ".txt")
        if label_path.exists():
            yield img_path, label_path
        else:
            print(f"[Uyarı] Etiket yok → {img_path.relative_to(images_dir)}", file=sys.stderr)

def rename_yolo_pairs(images_dir, labels_dir,
                      prefix="", suffix="",
                      zfill=4, start_idx=1, dry_run=False):
    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)

    if not images_dir.is_dir() or not labels_dir.is_dir():
        raise NotADirectoryError("images_dir ve/veya labels_dir bulunamadı!")

    pairs = list(collect_pairs(images_dir, labels_dir))
    if not pairs:
        raise RuntimeError("Eşleşen görüntü-etiket çifti bulunamadı.")

    for idx, (img_path, label_path) in enumerate(pairs, start=start_idx):
        num = str(idx).zfill(zfill)
        new_base = f"{prefix}{num}{suffix}"
        img_new = img_path.with_name(new_base + img_path.suffix)
        label_new = label_path.with_name(new_base + ".txt")

        # Çakışma kontrolü
        if img_new.exists() or label_new.exists():
            raise FileExistsError(f"Dosya zaten var: {img_new} | {label_new}")

        print(f"{img_path.name}  →  {img_new.name}")
        print(f"{label_path.name} →  {label_new.name}")

        if not dry_run:
            shutil.move(img_path, img_new)
            shutil.move(label_path, label_new)

    print("✅  Tamamlandı." + (" (dry-run)" if dry_run else ""))

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="YOLO görüntü-etiket çiftlerini birlikte yeniden adlandır."
    )
    parser.add_argument("--images_dir", required=True, help="images/ klasörü")
    parser.add_argument("--labels_dir", required=True, help="labels/ klasörü")
    parser.add_argument("--prefix", default="", help="Dosya adına önek")
    parser.add_argument("--suffix", default="", help="Dosya adına sonek")
    parser.add_argument("--zfill", type=int, default=4,
                        help="Numaradaki hane sayısı (vars: 4 → 0001)")
    parser.add_argument("--start_idx", type=int, default=1,
                        help="Numaralandırmaya kaçtan başlanacak?")
    parser.add_argument("--dry_run", action="store_true",
                        help="Önce deneme: sadece ne olacağını yaz, dosya taşıma!")
    args = parser.parse_args()

    rename_yolo_pairs(**vars(args))


usage: ipykernel_launcher.py [-h] --images_dir IMAGES_DIR --labels_dir
                             LABELS_DIR [--prefix PREFIX] [--suffix SUFFIX]
                             [--zfill ZFILL] [--start_idx START_IDX]
ipykernel_launcher.py: error: the following arguments are required: --images_dir, --labels_dir


SystemExit: 2

c:\Users\524ha\anaconda3\envs\plaka\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
